In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
import warnings
warnings.filterwarnings('ignore')

try:
    %cd openscope_databook
    from databook_utils.dandi_utils import dandi_download_open
    %cd ..
except:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .
        
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from math import sqrt
from scipy import interpolate

from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
from dandi import dandiapi
from pynwb import NWBHDF5IO
import scipy.signal
from scipy.ndimage import gaussian_filter
import icsd
from scipy.signal import welch
import math
import quantities as pq
import gc

%matplotlib inline

sub_sessions = [("sub-619296", "ses-1187930705"), ("sub-620333", "ses-1188137866"), ("sub-620334", "ses-1189887297"), ("sub-625545", "ses-1182865981"), ("sub-625554", "ses-1181330601"), ("sub-625555", "ses-1183070926"), ("sub-630506", "ses-1192952695"), ("sub-631510", "ses-1196157974"), ("sub-631570", "ses-1194857009"), ("sub-633229", "ses-1199247593"), ("sub-637484", "ses-1208667752")]

#read nwb file
session_num = 0
filepath = f"../material/00248_v240130/{sub_sessions[session_num][0]}/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}_ogen.nwb"
stim_io = NWBHDF5IO(filepath, mode="r", load_namespaces=True)
stim_nwb = stim_io.read()


C:\Users\USER\Desktop\minseok_kim\code\openscope_databook
C:\Users\USER\Desktop\minseok_kim\code


In [5]:
probe_num = 2

stim_num = [110000, 110101, 110105, 110106, 110107, 110109, 110110, 110111, 110506, 110511, 111105, 111109, 111201, 111299, 111301, 111302, 111303, 111304, 111305, 111306, 111307, 111308]
frame_stimtype = [(0, '0'), (3, 'IC1'), (7, 'IC2'), (4, 'LC1'), (6, 'LC2'), (8, 'IRE1'), (9, 'IRE2')]

lfp_filepath = f"../material/00248_v240130/{sub_sessions[session_num][0]}/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}_probe-" + str(probe_num) + "_ecephys.nwb"
lfp_io = NWBHDF5IO(lfp_filepath, mode="r", load_namespaces=True)
lfp_nwb = lfp_io.read()
lfp = lfp_nwb.acquisition["probe_" + str(probe_num) + "_lfp_data"]

In [6]:
for i in range(len(lfp.electrodes)) :
    print(i, lfp.electrodes['location'][i], round(lfp.electrodes['probe_vertical_position'][i]/1000, 3), round(lfp.electrodes['y'][i]/1000, 3))

0 SCig 0.04 3.111
1 SCig 0.08 3.076
2 SCig 0.12 3.04
3 SCig 0.16 2.996
4 SCig 0.2 2.961
5 SCig 0.24 2.925
6 SCig 0.28 2.89
7 SCig 0.32 2.854
8 SCig 0.36 2.819
9 SCig 0.4 2.775
10 SCig 0.44 2.739
11 SCig 0.48 2.704
12 SCig 0.52 2.668
13 SCig 0.56 2.633
14 SCig 0.6 2.597
15 SCig 0.64 2.553
16 SCig 0.68 2.518
17 SCig 0.72 2.483
18 SCig 0.76 2.447
19 POST 0.8 2.421
20 POST 0.84 2.385
21 POST 0.88 2.359
22 POST 0.92 2.323
23 POST 0.96 2.297
24 POST 1.0 2.27
25 POST 1.04 2.234
26 POST 1.08 2.208
27 POST 1.12 2.172
28 POST 1.16 2.145
29 POST 1.2 2.118
30 POST 1.24 2.081
31 POST 1.28 2.054
32 POST 1.32 2.017
33 HPF 1.36 1.989
34 HPF 1.4 1.962
35 HPF 1.44 1.924
36 HPF 1.48 1.896
37 HPF 1.52 1.858
38 HPF 1.56 1.83
39 HPF 1.6 1.801
40 HPF 1.64 1.763
41 HPF 1.68 1.734
42 HPF 1.72 1.695
43 HPF 1.76 1.666
44 HPF 1.8 1.637
45 HPF 1.84 1.598
46 HPF 1.88 1.568
47 VISp6b 1.92 1.529
48 VISp6a 1.96 1.5
49 VISp6a 2.0 1.46
50 VISp6a 2.04 1.431
51 VISp6a 2.08 1.401
52 VISp6a 2.12 1.362
53 VISp5 2.16 1.333
54

In [7]:
def get_all_stim_times(stim_nwb, frame_num):
    stim_table = stim_nwb.intervals["ICwcfg1_presentations"] #chosen stimulus

    frame = stim_table.frame[:]
    nzframe_idxs = np.nonzero(frame)[0]
    start_idx = int(nzframe_idxs[0])
    end_idx = int(nzframe_idxs[-1])

    start_time = stim_table.start_time[start_idx]
    end_time = stim_table.stop_time[end_idx]

    stim_select = lambda row : int(row['frame'].iloc[0]) == frame_num
    all_stim_times = [float(row['start_time'].iloc[0]) for i, row in enumerate(stim_table) if i % 2 == 0 and stim_select(row) and i >= start_idx and i <= end_idx]
    print(len(all_stim_times))
    return all_stim_times


def get_timestamps_data(lfp, all_stim_times):
    #print("First timestamp stimulus data: ", all_stim_times[0])
    #print("Last timestamp stimulus data: ", all_stim_times[-1])
    #print("First timestamp LFP data: ", lfp.timestamps[0])
    #print("Last timestamp LFP data: ", lfp.timestamps[-1])
    period_start = lfp.timestamps[0]
    period_end = lfp.timestamps[-1]
    # filter stim_timestamps to just timestamps within period
    stim_times = np.array([ts for ts in all_stim_times if ts >= period_start and ts <= period_end])
    if len(stim_times) == 0:
        raise ValueError("There are no stimulus timestamps in that period")

    # find indices within lfp data that correspond to period bounds
    period_start_idx, period_end_idx = None, None
    for i, ts in enumerate(lfp.timestamps):
        if not period_start_idx and ts >= period_start:
            period_start_idx = i
        if period_start_idx and ts >= period_end:
            period_end_idx = i
            break

    if period_start_idx == None or period_end_idx == None:
        raise ValueError("Period bounds not found within lfp data")
    #print(period_start_idx, lfp.timestamps[period_start_idx])
    # get slice of LFP data corresponding to the period bounds
    lfp_timestamps = lfp.timestamps[period_start_idx:period_end_idx]
    lfp_data = lfp.data[period_start_idx:period_end_idx]
    return lfp_timestamps, lfp_data, stim_times

def get_interp_lfp(interp_hz, lfp_timestamps, lfp_data) :
    
    # generate regularly-space x values and interpolate along it
    time_axis = np.arange(lfp_timestamps[0], lfp_timestamps[-1], step=(1/interp_hz))
    interp_channels = []

    # interpolate channel by channel to save RAM
    for channel in range(lfp_data.shape[1]):
        f = interpolate.interp1d(lfp_timestamps, lfp_data[:,channel], axis=0, kind="nearest", fill_value="extrapolate")
        interp_channels.append(f(time_axis))

    interp_lfp = np.transpose(interp_channels)
    return interp_lfp

# def lfp_smoothing(lfp_data, bin_size) :
#     S = np.zeros(lfp_data.shape)
#     for t in range(lfp_data.shape[0]) :
#         if t< bin_size :
#             S[t] = np.mean(lfp_data[:t+1])
#         else :
#             S[t] = np.sum(lfp_data[t-bin_size:t])/bin_size
#     return S

def get_windows(window_start_time, window_end_time, interp_hz, stim_times, lfp_timestamps, interp_lfp) :
    # validate window bounds
    if window_start_time > 0:
        raise ValueError("start time must be non-positive number")
    if window_end_time <= 0:
        raise ValueError("end time must be positive number")

    # get event windows
    windows = []
    window_length = int((window_end_time-window_start_time) * interp_hz)

    for stim_ts in stim_times:
        # convert time to index
        #print(stim_ts)
        start_idx = int( (stim_ts + window_start_time - lfp_timestamps[0]) * interp_hz )
        end_idx = start_idx + window_length

        # bounds checking
        if start_idx < 0 or end_idx > len(interp_lfp):
            print("bound out", start_idx, end_idx)
            continue

        windows.append(interp_lfp[start_idx:end_idx])

    if len(windows) == 0:
        raise ValueError("There are no windows for these timestamps")

    windows = np.array(windows)
    return windows

interp_hz = 1000
stim_window_start_time = -0.0
stim_window_end_time = 0.4
stim_windows = []

for frame_num in range(len(stim_num)) :
    print(frame_num)
    all_stim_times_temp = get_all_stim_times(stim_nwb, frame_num)
    lfp_timestamps_temp, lfp_data_temp, stim_times_temp = get_timestamps_data(lfp, all_stim_times_temp)
    interp_lfp_temp = get_interp_lfp(interp_hz, lfp_timestamps_temp, lfp_data_temp)
    #interp_lfp_temp = lfp_smoothing(interp_lfp_temp, 25)
    stim_windows.append(get_windows(stim_window_start_time, stim_window_end_time, interp_hz, stim_times_temp, lfp_timestamps_temp, interp_lfp_temp))
    
print("gc")
del stim_nwb
stim_io.close()
del stim_io
gc.collect()


0
400
1
400
2
400
3
400
4
400
5
400
6
400
7
400
8
50
9
50
10
400
11
400
12
400
13
400
14
50
15
50
16
50
17
50
18
50
19
50
20
50
21
50
gc


38293

In [8]:
#save LFP
os.makedirs(f'../material/LFP_npy_data/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}', exist_ok = True)
for frame_num in range(len(stim_num)) : 
    np.save(f'../material/LFP_npy_data/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}/probe{probe_num}_frame{frame_num}', stim_windows[frame_num])

In [ ]:
import numpy as np
import gc
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sn 
import pandas as pd
import matplotlib.pyplot as plt

# 데이터셋 클래스
class CSDDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32).unsqueeze(1).to(device)  # (200, 400, 84) -> (200, 1, 400, 84)
        self.labels = torch.tensor(labels, dtype=torch.long).to(device)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 개선된 CNN 모델 정의
class CNN(nn.Module):
    def __init__(self, kernel_size1, kernel_size2, kernel_size3):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=kernel_size1, padding=kernel_size1//2)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=kernel_size2, padding=kernel_size2//2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=kernel_size2, padding=kernel_size2//2)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=1)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)
        #self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(8638720, 256)
        self.fc2 = nn.Linear(256, len(frame_stimtype))

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)  # flatten
        x = self.relu(self.fc1(x)) #x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

def train_and_evaluate(train_loader, val_loader, model, criterion, optimizer, num_epochs=200):
    post_inference_memory = torch.cuda.memory_allocated()
    print(f"After inference: {post_inference_memory / (1024 ** 2):.2f} MB")
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        correct = 0

        y_pred = []
        y_true = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                pred = outputs.argmax(dim=1, keepdim=True)
                correct += pred.eq(labels.view_as(pred)).sum().item()
                
        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')
        
    # y_pred = []
    # y_true = []
    
    # with torch.no_grad():
    #     for inputs, labels in val_loader:
    #         outputs = model(inputs)
    #         outputs = (torch.max(torch.exp(outputs), 1)[1]).data.cpu().numpy()
    #         y_pred.extend(outputs)
            
    #         labels = labels.data.cpu().numpy()
    #         y_true.extend(labels)

    # classes = [stimtype for frame_num, stimtype in frame_stimtype]

    # cf_matrix = confusion_matrix(y_true, y_pred)
    # df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
    #                  columns = [i for i in classes])
    # plt.figure(figsize = (12,7))
    # sn.heatmap(df_cm, annot=True)
    # plt.savefig('output.png')

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)
    plt.figure(figsize=(10, 8))
    sn.heatmap(cm, annot=True, fmt='.2f' if normalize else 'd', cmap=cmap, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    #plt.savefig(f'../material/LFP_CNN/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}_IC_IRE_3hl_only_V1.jpg', format='jpeg')
    plt.savefig(f'../material/LFP_CNN/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}_IC_IRE_3hl_V1_LM.jpg', format='jpeg')
    
    plt.show()
    

def evaluate_model_on_test_set(model, test_loader, class_names, all_preds, all_labels):
    model.eval()
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds = np.append(all_preds, preds.cpu().numpy())
            all_labels = np.append(all_labels, labels.cpu().numpy())
    
    return all_preds, all_labels

In [1]:
import json
#load LFP data
stim_num = [110000, 110101, 110105, 110106, 110107, 110109, 110110, 110111, 110506, 110511, 111105, 111109, 111201, 111299, 111301, 111302, 111303, 111304, 111305, 111306, 111307, 111308]
sub_sessions = [("sub-619296", "ses-1187930705"), ("sub-620333", "ses-1188137866"), ("sub-620334", "ses-1189887297"), ("sub-625545", "ses-1182865981"), ("sub-625554", "ses-1181330601"), ("sub-625555", "ses-1183070926"), ("sub-630506", "ses-1192952695"), ("sub-631510", "ses-1196157974"), ("sub-631570", "ses-1194857009"), ("sub-633229", "ses-1199247593"), ("sub-637484", "ses-1208667752")]
valid_session_nums = [2, 3, 6, 7, 9, 10]
all_confusion_matrix = {} # 
for session_num in valid_session_nums:
    stim_windows = [] # (trial, time, channel)
    probe_num = 2
    stim_windows2 = []
    probe_num2 = 3
    for frame_num in range(len(stim_num)) : 
        stim_windows.append(np.load(f'../material/LFP_npy_data/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}/probe{probe_num}_frame{frame_num}.npy'))
        stim_windows2.append(np.load(f'../material/LFP_npy_data/{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}/probe{probe_num2}_frame{frame_num}.npy'))
    
    ################################################################################################
    #data(only V1)
    frame_stimtype = [(3, 'IC1'), (7, 'IC2'), (8, 'IRE1'), (9, 'IRE2')]
    
    data = [] # (trial #, channel, time)
    labels = []
    
    for i, (frame_num, stimtype) in enumerate(frame_stimtype) :
        trial_nums = np.arange(stim_windows[frame_num].shape[0])
        random.shuffle(trial_nums)
        for j, trial_num in enumerate(trial_nums) :
            if j == 50 : break
            temp_data = np.concatenate((stim_windows[frame_num][trial_num,:,:], stim_windows2[frame_num][trial_num,:,:]), axis = 1)
            data.append(temp_data)
            labels.append(i)
    data = np.array(data)
    labels = np.array(labels)
    del stim_windows
    del stim_windows2
    gc.collect()
    ################################################################################################
    # #data(V1 + LM)
    # import random
    # frame_stimtype = [(3, 'IC1'), (7, 'IC2'), (8, 'IRE1'), (9, 'IRE2')]
    
    # data = [] # (trial #, channel, time)
    # labels = []
    
    # for i, (frame_num, stimtype) in enumerate(frame_stimtype) :
    #     trial_nums = np.arange(stim_windows[frame_num].shape[0])
    #     random.shuffle(trial_nums)
    #     for j, trial_num in enumerate(trial_nums) :
    #         if j == 50 : break
    #         data.append(stim_windows[frame_num][trial_num,:,:])
    #         labels.append(i)
    # data = np.array(data)
    # labels = np.array(labels)
    # del stim_windows
    # del stim_windows2
    # gc.collect()
    ################################################################################################
    
    
    if __name__ == '__main__' :
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(device)
        
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    class_names = [stimtype for frame_num, stimtype in frame_stimtype]
    
    kernel_sizes = [
        (3, 3, 3),  
    ]
    for kernel_size1, kernel_size2, kernel_size3 in kernel_sizes:
        print(f'{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}')
        print(f'Using kernel sizes: conv1={kernel_size1}, conv2={kernel_size2}, conv3={kernel_size3}')
        
        all_preds = np.array([])
        all_labels = np.array([])
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(data)): # cross validation
            print(f'Fold {fold+1}')
            train_data, val_data = data[train_idx], data[val_idx]
            train_labels, val_labels = labels[train_idx], labels[val_idx]
            
            train_dataset = CSDDataset(train_data, train_labels)
            val_dataset = CSDDataset(val_data, val_labels)
            
            train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)
            
            model = CNN(kernel_size1, kernel_size2, kernel_size3).to(device)
            criterion = nn.CrossEntropyLoss() #pytorch의 crossentropy에는 softmax가 포함됨
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            
            train_and_evaluate(train_loader, val_loader, model, criterion, optimizer)
            all_preds, all_labels = evaluate_model_on_test_set(model, val_loader, class_names, all_preds, all_labels)
        cm = confusion_matrix(all_labels, all_preds)
        plot_confusion_matrix(cm, class_names)
    all_confusion_matrix.append([f'{sub_sessions[session_num][0]}_{sub_sessions[session_num][1]}', cm])
os.makedirs(f'../material/LFP_CNN', exist_ok = True)
np.save(f'../material/LFP_CNN/all_confusion_matrix.npy', all_confusion_matrix)

[[3, 4]]


cuda
sub-630506_ses-1192952695
Using kernel sizes: conv1=3, conv2=3, conv3=3
Fold 1
After inference: 8489.38 MB
Epoch 1/200, Validation Loss: 9.9164, Accuracy: 0.2500
Epoch 2/200, Validation Loss: 8.4445, Accuracy: 0.1500
Epoch 3/200, Validation Loss: 2.5457, Accuracy: 0.2250
Epoch 4/200, Validation Loss: 3.5367, Accuracy: 0.2250
Epoch 5/200, Validation Loss: 3.7726, Accuracy: 0.2500
Epoch 6/200, Validation Loss: 3.6790, Accuracy: 0.2250
Epoch 7/200, Validation Loss: 1.4018, Accuracy: 0.1500
Epoch 8/200, Validation Loss: 0.9296, Accuracy: 0.1500
Epoch 9/200, Validation Loss: 1.2089, Accuracy: 0.2250
Epoch 10/200, Validation Loss: 1.2721, Accuracy: 0.1500
Epoch 11/200, Validation Loss: 0.9940, Accuracy: 0.2750
Epoch 12/200, Validation Loss: 1.7608, Accuracy: 0.1750
Epoch 13/200, Validation Loss: 3.1614, Accuracy: 0.3000
Epoch 14/200, Validation Loss: 2.6238, Accuracy: 0.2000
Epoch 15/200, Validation Loss: 3.9950, Accuracy: 0.3000
Epoch 16/200, Validation Loss: 4.6001, Accuracy: 0.3000
E

In [6]:
import torch

# GPU 메모리 초기 상태
initial_memory = torch.cuda.memory_allocated()
print(f"Initial GPU memory usage: {initial_memory / (1024 ** 2):.2f} MB")

# 모델 생성 후 GPU 메모리 확인
model = CNN(3, 3).to(device)
post_model_memory = torch.cuda.memory_allocated()
print(f"After model allocation: {post_model_memory / (1024 ** 2):.2f} MB")

# 입력 데이터 예시
example_input = torch.randn(1, 1, 400, 168).to(device)

# 모델을 사용한 후 GPU 메모리 확인
_ = model(example_input)
post_inference_memory = torch.cuda.memory_allocated()
print(f"After inference: {post_inference_memory / (1024 ** 2):.2f} MB")

Initial GPU memory usage: 4320.75 MB
After model allocation: 12579.34 MB
After inference: 8449.26 MB
